#### Librerías y Parámetros

In [1]:
"""
Integra todos los pasos del proceso de cálculo de la PCR CP y realiza los cálculos, exporta el output
"""

import src.parametros as p
import src.aux_tools as aux_tools
import src.prep_insumo as prep_data
import src.devenga as devg
import src.fluctuacion as fluc
import src.deterioro as det
import src.mapeo_contable as mapcont
import polars as pl


# lee los parámetros e insumos relevantes
FECHA_VALORACION = p.FECHA_VALORACION
FECHA_VALORACION_ANTERIOR = p.FECHA_VALORACION_ANTERIOR
FECHA_TRANSICION = p.FECHA_TRANSICION

In [2]:
def run_pcr():
    # Lectura de insumos
    # Insumos transversales
    param_contab = pl.read_excel(
        p.RUTA_INSUMOS, sheet_name=p.HOJA_PARAMETROS_CONTAB
    ).filter(
        pl.col("estado_insumo") == 1
    )  # Solo se usan las configuraciones activas (1)
    excepciones = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_EXCEPCIONES_50_50)
    gasto = pl.read_excel(p.RUTA_GASTOS)
    tasa_cambio = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_MONEDA)
    descuentos = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_DESCUENTO)
    # El diccionario o tabla de correspondencia de outputs con entradas contables
    input_map_bts = pl.read_excel(p.RUTA_REL_BT, infer_schema_length=2000).filter(
        ~pl.col("clasificacion_adicional").is_in(["MAT", "REC"])
    )
    input_tipo_seguro = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_TIPO_SEGURO)
    tabla_nomenclatura = pl.read_excel(p.RUTA_NOMENCLATURA, sheet_name="V2")
    # Insumos de recibos contabilizados en SAP
    produccion_dir = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_PDN)
    componente_inversion = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_COMP_INV)
    cesion_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_CESION)
    comision_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_COMISION_REA)
    costo_contrato_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_COSTO_CONTRATO)
    seguimiento_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_SEGUIMIENTO_REA)
    produccion_arl = pl.read_excel(p.RUTA_PRODUCCION_ARL)
    costo_contrato_arl = pl.read_excel(p.RUTA_COSTO_CONTRATO_ARL)
    camara_soat = pl.read_excel(p.RUTA_CAMARA_SOAT)
    # Insumos de onerosidad leidos desde el datalake
    onerosidad = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_ONEROSIDAD)
    recup_onerosidad = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_RECUP_ONEROSIDAD)
    # Insumos de riesgo de credito cargado por equipo de riesgo financiero
    riesgo_credito = pl.read_excel(p.RUTA_RIESGO_CREDITO)
    # Insumos no devengables
    cartera = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_CARTERA)
    cartera_arl = pl.read_excel(p.RUTA_CARTERA_ARL)
    cuenta_corriente = pl.read_excel(p.RUTA_CUENTA_CORRIENTE)
    cuenta_corriente_arl = pl.read_excel(p.RUTA_CUENTA_CORRIENTE_ARL)

    produccion_arl_prep = prep_data.prep_input_produccion_arl(produccion_arl)
    produccion_dir = pl.concat(
        [produccion_dir, produccion_arl_prep], how="diagonal_relaxed"
    ).with_columns(
        pl.col('poliza').cast(pl.Utf8)
    )

    costo_contrato_rea = pl.concat(
        [costo_contrato_rea, costo_contrato_arl], how="diagonal_relaxed"
    )

    cartera = pl.concat(
        [
            cartera,
            cartera_arl.with_columns(
                fecha_expedicion_poliza=pl.col("mes_cotizacion").dt.month_start()
            ),
        ],
        how="diagonal_relaxed",
    )

    cuenta_corriente = pl.concat(
        [cuenta_corriente, cuenta_corriente_arl], how="diagonal_relaxed"
    )

    # Prepara cada insumo para entrar a devengo
    insumos_devengo = [
        # prepara insumos seguro directo
        prep_data.prep_input_prima_directo(
            produccion_dir, param_contab, excepciones, FECHA_VALORACION
        ),
        prep_data.prep_input_dcto_directo(
            produccion_dir, param_contab, excepciones, descuentos, FECHA_VALORACION
        ),
        prep_data.prep_input_gasto_directo(
            produccion_dir, param_contab, excepciones, gasto, FECHA_VALORACION
        ),
        prep_data.prep_input_componente_inversion(
            componente_inversion, param_contab, FECHA_VALORACION
        ),
        prep_data.prep_input_onerosidad(
            onerosidad, param_contab, FECHA_VALORACION
        ),
        prep_data.prep_input_prima_directo(
            camara_soat, param_contab, excepciones, FECHA_VALORACION
        ),
        # prepara insumos reaseguro
        prep_data.prep_input_prima_rea(
            cesion_rea, param_contab, excepciones, FECHA_VALORACION
        ),
        prep_data.prep_input_dcto_rea(
            cesion_rea, param_contab, excepciones, descuentos, FECHA_VALORACION
        ),
        prep_data.prep_input_gasto_rea(
            cesion_rea, param_contab, excepciones, gasto, FECHA_VALORACION
        ),
        prep_data.prep_input_comi_rea(
            comision_rea, param_contab, excepciones, FECHA_VALORACION
        ),
        prep_data.prep_input_costo_con(
            costo_contrato_rea,
            seguimiento_rea,
            param_contab,
            excepciones,
            FECHA_VALORACION,
        ),
        prep_data.prep_input_recup_onerosidad_pp(
            onerosidad, cesion_rea, param_contab, excepciones, FECHA_VALORACION
        ),
        prep_data.prep_input_recup_onerosidad_np(
            recup_onerosidad,
            seguimiento_rea,
            param_contab,
            excepciones,
            FECHA_VALORACION,
        ),
    ]

    input_consolidado = pl.concat(
        aux_tools.alinear_esquemas(insumos_devengo), how="diagonal"
    )
    
    # devuelve la base ya devengada, con las columnas de movimientos saldos y de fluctuación
    output_devengo_fluct = (
        devg.devengar(input_consolidado, FECHA_VALORACION)
        .pipe(fluc.calc_fluctuacion, tasa_cambio)
        .pipe(det.calc_deterioro, riesgo_credito, FECHA_VALORACION)
    )
    output_devengo_fluct.write_excel(p.RUTA_SALIDA_DEVENGO)

    # Insumos no devengables
    insumos_no_devengo = [
        prep_data.prep_input_cartera(cartera, param_contab, FECHA_VALORACION),
        prep_data.prep_input_cartera(cuenta_corriente, param_contab, FECHA_VALORACION),
    ]

    # convierte a output contable
    output_contable = mapcont.gen_output_contable(
        output_devengo_fluct,
        input_map_bts,
        input_tipo_seguro,
        tabla_nomenclatura,
        insumos_no_devengo,
    ).pipe(mapcont.agregar_marca_onerosidad, onerosidad, FECHA_VALORACION)
    output_contable.write_excel(p.RUTA_SALIDA_CONTABLE)

    return output_devengo_fluct, output_contable

In [3]:
output_devengo = run_pcr()[1]
output_devengo

Could not determine dtype for column 1, falling back to string
Could not determine dtype for column 4, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 0, falling back to string


Registros antes del cruce BT:  72
Registros despues del cruce BT:  72


aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,numero_documento_sap,amparo,cdsubgarantia,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,uen,mes_cotizacion,clasificacion_adicional,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,podto_comercial,podto_tecnico,tipo_gasto,porc_gasto,prioridad_match,rn,fecha_cancelacion_poliza,fecha_inicio_vigencia_poliza,…,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,porc_consumo_limite,estado_devengo,valor_liberacion_acum,tasa_cambio_fecha_valoracion,tasa_cambio_fecha_valoracion_anterior,tasa_cambio_fecha_constitucion,prob_incumplimiento_actual,prob_incumplimiento_anterior,cambio_prob_incumplimiento,tipo_movimiento,valor_md,valor_ml,fecha_corte,cartera_riesgo,concepto,tipo_movimiento_codigo,indicativo_periodo_movimiento_codigo,concepto_codigo,clasificacion_adicional_codigo,tipo_negocio_codigo,tipo_reaseguro_codigo,tipo_reasegurador_codigo,compania_codigo,tipo_contabilidad_codigo,transicion_codigo,tipo_reserva,tipo_seguro,tipo_seguro_codigo,naturaleza,bt,descripcion_bt,onerosidad
str,str,str,str,date,i64,i64,str,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,i64,date,str,str,i64,i32,date,f64,f64,str,f64,i32,i64,date,date,…,i64,f64,i32,i32,i32,f64,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,date,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
null,"""componente_inversion_directo""","""directo""","""BAN110581505""",2025-05-16,10076864615,10076864615,"""SBV""","""NDX """,1,"""02""","""081""","""004""","""NDX""","""30""","""COP""",2025-09-07,2025-08-16,2025-09-16,null,null,-8649.0,null,null,"""componente_inversion""","""poliza""",-1,null,null,null,null,null,null,null,null,2025-05-22,2025-05-16,…,null,null,202509,null,null,null,null,-8649.0,1.0,1.0,1.0,null,null,null,"""valor_constitucion""",8649.0,8649.0,null,null,null,"""CO""","""NA""",null,null,"""D""","""NA""","""NA""","""2000""","""NIIF17C""","""NO""","""PCR_CP""","""Personas""","""P""",null,null,null,"""NO"""
null,"""componente_inversion_directo""","""directo""","""BAN110581505""",2025-05-16,10076864615,10076864615,"""SBV""","""NDX """,1,"""02""","""081""","""004""","""NDX""","""30""","""COP""",2025-09-07,2025-08-16,2025-09-16,null,null,-8649.0,null,null,"""componente_inversion""","""poliza""",-1,null,null,null,null,null,null,null,null,2025-05-22,2025-05-16,…,null,null,202509,null,null,null,null,-8649.0,1.0,1.0,1.0,null,null,null,"""valor_constitucion""",8649.0,8649.0,null,null,null,"""CO""","""NA""",null,null,"""D""","""NA""","""NA""","""2000""","""NIIF17L""","""NO""","""PCR_CP""","""Personas""","""P""",null,null,null,"""NO"""
null,"""componente_inversion_directo""","""directo""","""BAN110581505""",2025-05-16,10076864615,10076864615,"""SBV""","""NDX """,1,"""02""","""081""","""004""","""NDX""","""30""","""COP""",2025-09-07,2025-08-16,2025-09-16,null,null,-8649.0,null,null,"""componente_inversion""","""poliza""",-1,null,null,null,null,null,null,null,null,2025-05-22,2025-05-16,…,null,null,202509,null,null,null,null,-8649.0,1.0,1.0,1.0,null,null,null,"""valor_constitucion""",8649.0,8649.0,null,null,null,"""CO""","""NA""",null,null,"""D""","""NA""","""NA""","""2000""","""NIIF4""","""NO""","""PCR_CP""","""Personas""","""P""",null,null,null,"""NO"""
null,"""componente_inversion_directo""","""directo""","""BAN110581505""",2025-05-16,10074557764,10074557764,"""SBV""","""NDX """,1,"""02""","""081""","""004""","""NDX""","""30""","""COP""",2025-09-07,2025-06-16,2025-07-16,null,null,-8649.0,null,null,"""componente_inversion""","""poliza""",-1,null,null,null,null,null,null,null,null,2025-05-22,2025-05-16,…,null,null,202509,null,null,null,null,-8649.0,1.0,1.0,1.0,null,null,null,"""valor_constitucion""",8649.0,8649.0,null,null,nul

In [4]:
output_devengo.filter(pl.col('tipo_insumo') == 'componente_inversion_directo').write_clipboard()